In [1]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

C:\Users\kimji\AppData\Local\Temp\ipykernel_13200\2284917278.py:14: FutureWarning: set_caching_enabled is deprecated and will be removed in the next major version of datasets. Use datasets.enable_caching() or datasets.disable_caching() instead. This function will be removed in a future version of datasets.
  set_caching_enabled(False)


In [2]:
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

In [4]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("PyTorch 버전:", torch.__version__)
print("CUDA 버전:", torch.version.cuda)
print("CUDA 사용 가능:", torch.cuda.is_available())
print("cuDNN 사용 가능:", torch.backends.cudnn.enabled)

if torch.cuda.is_available():
    print("GPU 수:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i} 이름:", torch.cuda.get_device_name(i))
else:
    print("GPU를 찾을 수 없습니다. CPU를 사용합니다.")

print(device)
print("cuDNN 버전:", torch.backends.cudnn.version())

PyTorch 버전: 1.12.1+cu116
CUDA 버전: 11.6
CUDA 사용 가능: True
cuDNN 사용 가능: True
GPU 수: 1
GPU 0 이름: NVIDIA GeForce RTX 2060
cuda
cuDNN 버전: 8302


In [5]:
transformers.logging.get_verbosity()

0

In [6]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET

In [ ]:
df =pd.read_csv(".train.csv")
df["votes"] = df["up_votes"]-df["down_votes"]

#Add audio path and renaming columns
df["path"] = "../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR_dataset/noise_remove/"+df["ID"]+".wav"
df.rename(columns = {'transcription':'sentence'}, inplace = True)

train,test = train_test_split(df, test_size=0.005, random_state=42)
common_voice_train_1 = Dataset.from_pandas(train[3000:])
common_voice_train_2 = Dataset.from_pandas(train[0:3000])

common_voice_test = Dataset.from_pandas(test)